In [1]:
!pip install kafka-python boto3 pyarrow

In [1]:
import json
import boto3
import pyarrow as pa
import pyarrow.parquet as pq
from kafka import KafkaConsumer
import uuid
import datetime

In [ ]:
# Konfigurasi AWS
s3 = boto3.client('s3')
BUCKET = "finnhub-stock-market"

# Kafka Consumer
consumer = KafkaConsumer(
    "stock_stream",
    bootstrap_servers=[":9092"],
    auto_offset_reset="earliest",
    enable_auto_commit=True,
    group_id="stock_consumer",
    value_deserializer=lambda v: json.loads(v.decode("utf-8"))
)

batch = []

def upload_parquet(records):
    """Upload batch records ke S3 sebagai file Parquet unik"""

    table = pa.Table.from_pylist(records)

    # ======== Nama File Unik ===========
    timestamp = datetime.datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    unique_id = uuid.uuid4().hex[:8]
    file_name = f"stock_data/trades_{timestamp}_{unique_id}_{table.num_rows}.parquet"
    # ===================================

    local_path = "/tmp/output.parquet"

    # buat file parquet
    pq.write_table(table, local_path)

    # upload ke S3
    s3.upload_file(local_path, BUCKET, file_name)
    print(f"Uploaded to S3: s3://{BUCKET}/{file_name}")

print("Consumer running...")

for msg in consumer:
    data = msg.value
    batch.append(data)

    # upload setiap 200 record
    if len(batch) >= 200:
        upload_parquet(batch)
        batch = []

Consumer running...


C:\Users\User\AppData\Local\Temp\ipykernel_14744\3217407254.py:23: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.datetime.utcnow().strftime("%Y%m%d_%H%M%S")


Uploaded to S3: s3://finnhub-stock-market/stock_data/trades_20251124_151800_dd936e5a_200.parquet
Uploaded to S3: s3://finnhub-stock-market/stock_data/trades_20251124_151901_bb7aa701_200.parquet
